In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Rounak\\Desktop\\OneDrive\\College\\Projects\\Severity-of-Toxic-Commentis-End-to-End\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [20]:
from src.SeverityOfToxicCommentsEndToEnd.constants import *
from src.SeverityOfToxicCommentsEndToEnd.utils.common import read_yaml, create_directories

In [21]:
print(CONFIG_FILE_PATH, PARAMS_FILE_PATH)

config\config.yaml params.yaml


In [22]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        print(data_ingestion_config.source_URL)
        return data_ingestion_config


In [23]:
import os
import urllib.request as request
import zipfile
from SeverityOfToxicCommentsEndToEnd.logging import logger
from SeverityOfToxicCommentsEndToEnd.utils.common import get_size 

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url = self.config.source_URL, filename = self.config.local_data_file)
            logger.info(f"{filename} downloaded successfully with the following info: {headers}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-09 19:13:33,709: INFO: common] Successfully read yaml file from config\config.yaml
[2023-07-09 19:13:33,711: INFO: common] Successfully read yaml file from params.yaml
[2023-07-09 19:13:33,712: INFO: common] Created directory at: artifacts
[2023-07-09 19:13:33,714: INFO: common] Created directory at: artifacts/data_ingestion
https://github.com/rounaccc/Severity-of-Toxic-Comments-End-to-End/raw/main/ToxicCommentsData.zip
[2023-07-09 19:13:48,719: INFO: 1737986185] artifacts/data_ingestion/ToxicCommentsData.zip downloaded successfully with the following info: Connection: close
Content-Length: 28414360
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9d1ff736c85b66d5762d14d3f896fd686b6fb01f7d2004b8e1828dff212833d0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8DEA:28D4:F26AD:173F29: